# ENGR 240: Spline Interpolation with SciPy

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%206/Worksheet%206-2%20Spline%20Interpolation%20with%20SciPy.ipynb)

## Introduction

In many engineering applications, we need to interpolate between discrete data points to create a smooth, continuous representation that approximates underlying behavior. Splines are piecewise polynomial functions that provide smooth interpolation between data points and are widely used in engineering design, data analysis, and computer graphics.

In this worksheet, we'll explore spline interpolation using SciPy's `interpolate` module, specifically focusing on cubic splines, which are among the most common types of splines used in engineering applications.

### Learning Objectives
- Understand the basic theory behind spline interpolation
- Implement various cubic spline end conditions using SciPy
- Extract and analyze spline coefficients
- Apply spline interpolation to a real engineering problem

### Application Context: Structural Beam Deflection Analysis

In this worksheet, we'll focus on a practical engineering application: analyzing the deflection of a beam under various loading conditions. Structural engineers often use spline interpolation to:

1. Smoothly interpolate between measured deflection points
2. Create continuous mathematical representations of physical measurements
3. Estimate values between measurement points
4. Generate smooth visualization of deformation profiles

The ability to properly implement various end conditions is especially important in beam analysis, as they represent different physical constraints (e.g., fixed ends, free ends, or specific known slopes).

## Theoretical Background: Spline Interpolation

### What is a Spline?

A spline is a piecewise polynomial function that smoothly connects a set of data points. The term "spline" originated from the flexible drafting tools used by shipbuilders and draftsmen to draw smooth curves through a set of points.

### Cubic Splines

Cubic splines are particularly popular because they provide a good balance between smoothness and computational efficiency. A cubic spline consists of third-degree polynomial segments connecting pairs of data points, where the polynomials are constrained to ensure the resulting curve is continuous and smooth.

For a set of data points $(x_0, y_0), (x_1, y_1), \ldots, (x_n, y_n)$ where $x_0 < x_1 < \ldots < x_n$, a cubic spline $S(x)$ is defined as:

$$S(x) = S_i(x) \quad \text{for } x_i \leq x \leq x_{i+1}, i = 0, 1, \ldots, n-1$$

where each $S_i(x)$ is a cubic polynomial of the form:

$$S_i(x) = a_i + b_i(x-x_i) + c_i(x-x_i)^2 + d_i(x-x_i)^3$$

The coefficients $a_i$, $b_i$, $c_i$, and $d_i$ are determined by enforcing the following constraints:

1. **Interpolation**: The spline passes through all data points
   - $S_i(x_i) = y_i$
   - $S_i(x_{i+1}) = y_{i+1}$
   
2. **Continuity**: The first and second derivatives are continuous at interior knots
   - $S_i'(x_{i+1}) = S_{i+1}'(x_{i+1})$
   - $S_i''(x_{i+1}) = S_{i+1}''(x_{i+1})$

### End Conditions

The constraints above give us $4n - 2$ equations for $4n$ unknown coefficients. To fully determine the spline, we need two additional conditions, typically specified at the endpoints. Common end conditions include:

1. **Natural Spline**: Second derivatives at endpoints are set to zero
   - $S''(x_0) = S''(x_n) = 0$
   
2. **Clamped Spline**: First derivatives at endpoints are specified
   - $S'(x_0) = f'_0$ and $S'(x_n) = f'_n$
   
3. **Not-a-knot**: Third derivative is continuous at the second and second-to-last points
   - $S_0'''(x_1) = S_1'''(x_1)$
   - $S_{n-2}'''(x_{n-1}) = S_{n-1}'''(x_{n-1})$
   
4. **Periodic**: Function and derivatives at endpoints match (for cyclic data)
   - $S^{(k)}(x_0) = S^{(k)}(x_n)$ for $k = 0, 1, 2$

The choice of end conditions depends on the physical constraints of the problem and the behavior expected at the boundaries.

## Setup and Imports

Let's start by importing the necessary libraries:

In [ ]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

## Task 1: Basic Spline Interpolation

Let's start with a simple example of beam deflection data. Consider a simply-supported beam with a concentrated load at its center. We have measured the deflection at several points along the beam:

In [ ]:
# Define beam data
# x represents position along the beam (meters)
# y represents measured deflection (millimeters)
x = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
y = np.array([0.0, -5.2, -8.1, -9.0, -8.1, -5.2, 0.0])

# Plot the measured data points
plt.figure()
plt.plot(x, y, 'o', markersize=8, label='Measured deflection')
plt.xlabel('Position along beam (m)')
plt.ylabel('Deflection (mm)')
plt.title('Beam Deflection Measurements')
plt.grid(True)
plt.legend()
plt.show()

### Step 1: Create a Basic Cubic Spline

Let's create a basic cubic spline interpolation using SciPy's `CubicSpline` function with default settings (natural spline):

In [ ]:
# Create a cubic spline with default settings (natural spline)
cs_natural = interpolate.CubicSpline(x, y)

# Create a finer grid for smooth plotting
x_fine = np.linspace(0, 6, 200)
y_natural = cs_natural(x_fine)

# Plot the results
plt.figure()
plt.plot(x, y, 'o', markersize=8, label='Measured data')
plt.plot(x_fine, y_natural, '-', label='Natural cubic spline')
plt.xlabel('Position along beam (m)')
plt.ylabel('Deflection (mm)')
plt.title('Beam Deflection with Natural Cubic Spline')
plt.legend()
plt.grid(True)
plt.show()

### Step 2: Examine the Spline Coefficients

The `CubicSpline` object contains the coefficients for each polynomial segment. Let's extract and examine these coefficients:

In [ ]:
# Display the coefficients of the natural cubic spline
print("Cubic Spline Coefficients:")
print("\nCoefficients shape:", cs_natural.c.shape)
print("\nCoefficient array:\n", cs_natural.c)
print("\nKnots (x values):\n", cs_natural.x)

# Print coefficients for each segment in a more readable format
print("\nCoefficients for each polynomial segment:")
for i in range(len(x) - 1):
    print(f"\nSegment {i} ({x[i]} ≤ x ≤ {x[i+1]})")
    # Note: SciPy's CubicSpline stores coefficients in the order [d, c, b, a] for each segment
    # The polynomials are of the form: a + b(x-x_i) + c(x-x_i)^2 + d(x-x_i)^3
    a = cs_natural(x[i])  # The value at the left endpoint is the constant term
    b = cs_natural.c[2, i]
    c = cs_natural.c[1, i]
    d = cs_natural.c[0, i]
    print(f"  P{i}(x) = {a:.4f} + {b:.4f}(x-{x[i]}) + {c:.4f}(x-{x[i]})^2 + {d:.4f}(x-{x[i]})^3")